In [2]:
import numpy as np
from numpy import array
from numpy.linalg import norm
from scipy.linalg import expm
from numpy import dot
import plotly.express as px
import pandas as pd
from cogent3.maths.matrix_exponential_integration import expected_number_subs
import math
from clock_project.maths.evolutionary_rate import calculate_stationary_distribution, calculate_stationary_rate, matrix_calibration
from clock_project.simulation.magnitude_quantification import calculate_ENS

In [3]:
#testing rate matrices and nucleotide distributions
Q1 = array([[-1.4, 0.1, 0.4, 0.9], 
           [4.0, -6.9, 0.9, 2.0], 
           [6.3, 2.0, -11.3, 3.0], 
           [0.7,0.1, 0.2, -1]])

Q2 = np.array([[-8.35, 1.98, 4.58, 1.79], 
           [4.69, -6.76, 1.08, 0.99], 
           [0.05, 0.58, -3.24, 2.61], 
           [0.05,1.32, 3.70, -5.07]])

Q3= np.array([
    [-4.5,  2.0,  1.0,  1.5],
    [ 2.0, -3.5,  0.5,  1.0],
    [ 1.0,  0.5, -2.5,  1.0],
    [ 0.0,  1.0,  2.0, -3.0]
])

Q4 = np.array([[-1.707,  0.537,  0.306,  0.864],
       [ 0.249, -0.889 ,  0.116,  0.525 ],
       [ 0.038,  0.182, -0.555,  0.335],
       [ 0.203,  0.580,  0.234, -1.017]])

Q5 = np.array([[-1.87,  0.59,  0.33,  0.95],
       [ 0.27, -0.97,  0.13,  0.57],
       [ 0.04,  0.20, -0.60,  0.36],
       [ 0.23,  0.62,  0.25, -1.10]])

Q6 = np.array([[-4.56,  1.59,  2.33,  0.64],
       [ 0.55, -1.98,  1.13,  0.3],
       [ 0.73,  1.43, -4.52,  2.36],
       [ 0.15,  0.37,  1.8, -2.32]])

Q7 = np.array([[-0.7,  0.2,  0.1,  0.4],
       [ 0.0286, -0.7286,  0.6,  0.1],
       [ 0.0143,  0.6, -0.9143,  0.3],
       [ 0.08,  0.14,  0.42, -0.64]])

i = np.array([0.25, 0.25, 0.25, 0.25])

test_nst_array1 = np.array([0.3, 0.4, 0.2, 0.1])
test_nst_array2 = np.array([0.7, 0.1, 0.1, 0.1])
test_nst_array3 = np.array([0.05, 0.35, 0.35, 0.25])

Q_list = [Q1, Q2, Q3, Q4, Q5, Q6, Q7]


In [4]:
import json
valid_matrix_full_path = '/Users/gulugulu/Desktop/honours/data_local_2/valid_matrix_full.json'
valid_matrix_full = json.load(open(valid_matrix_full_path, 'r'))
matrices_pair_list = []
for gene, matrices in valid_matrix_full.items():
    matrices_pair_list.extend(matrices)
matrices_list = [np.array(matrix) for matrix_pair in matrices_pair_list for matrix in matrix_pair.values()]

i = 0
complex_eigens_matrix_list = []
for matrix in matrices_list:
    eigens = np.linalg.eig(matrix)[0]  # Extract eigenvalues
    if any(isinstance(eigen, complex) and eigen.imag != 0 for eigen in eigens):
        complex_eigens_matrix_list.append(matrix)
        i += 1

Stationary

In [5]:
#stationary process ENS accumulation function
def generate_ENS(Q1, Q2, t_range, t1):
    """
    Generates the ENS over a range of time points using two different Q matrices before and after a specified time point t1.
    
    Parameters:
    - pi: A numpy array of shape (1, 4) representing the vector pi.
    - Q1, Q2: Two numpy arrays of shape (4, 4) representing the original and new rate matrices.
    - t_range: numpy.linspace defining the start and end of the time range.
    - t1: The time point at which to switch from using Q1 to Q2.
    
    Returns:
    - A list of ENS values for each time point in the range.
    """
    pi = calculate_stationary_distribution(Q1)

    ens_values = []
    ens_accumulated = 0  # To keep track of the accumulated ENS value
    
    for t in t_range:
        if t <= t1:
            ens = -np.sum(pi * np.diag(Q1)) * t + ens_accumulated
            ens_values.append(ens)
        else:
            ens_2 = -np.sum(pi * np.diag(Q2)) * (t-t1) + ens # Update the accumulated ENS at t1 to continue from this point using Q2
            ens_values.append(ens_2)
    
    return ens_values

In [12]:
# Ploting the evolutionary rate mu over time with rate matrix switch in the middle
import plotly.graph_objects as go
t_range = np.linspace(0, 10, 999)
mu_value = []

# Generate mu values
for t in t_range:
    mu = calculate_stationary_rate(Q3*0.1)
    mu_value.append(mu)
    

# Create the plot
fig = go.Figure()

# Add Q3 line
fig.add_trace(go.Scatter(x=np.linspace(0, 10, 999), y=mu_value, mode='lines', line=dict(color='#58B8D1', width=4)))


# Update layout for better presentation
fig.update_layout(
    title=None,
    title_font=dict(size=20),  # Increase title size, make it bold
    xaxis_title='In-group JSD',
    xaxis=dict(
        title_font=dict(size=25, family='Latex'),
        tickfont=dict(size=20),
    ),
    yaxis_title=r'$\mu$',
    yaxis=dict(
        title_font=dict(size=25),
        tickfont=dict(size=20),
    ),
    legend=dict(
        title=None,
        font=dict(size=16),  # Adjust font size of legend
        x=1.02,  # Adjust position of legend
        y=1,
        bordercolor="Black",
        borderwidth=1
    ),
    template='plotly_white',
    width=800,
    height=450,
    margin=dict(l=50, r=50, t=50, b=50)
)

# fig.write_image("/Users/gulugulu/repos/PuningAnalysis/results/figures/mu_stationary.pdf")

# Show the plot
fig.show()

In [7]:



# Generate ENS values
ens_values = generate_ENS(Q1, Q1, np.linspace(0, 1, 999), 0)
# Create the plot
fig = go.Figure()

# Add Q3 line
fig.add_trace(go.Scatter(x=np.linspace(0, 10, 999), y=ens_values, mode='lines', line = dict(color = '#cd6d2e', width = 4)))

# Add Q5 line

# Update layout for better presentation
fig.update_layout(
    title=None,
    title_font=dict(size=20),  # Increase title size, make it bold
    xaxis_title='<b>Time</b>',
    xaxis=dict(
        title_font=dict(size=25),
        tickfont=dict(size=20),
    ),
    yaxis_title=r'$ENS$',
    yaxis=dict(
        title_font=dict(size=25),
        tickfont=dict(size=20),
    ),
    template='plotly_white',
        width=800,
        height=450,
    margin=dict(l=50, r=50, t=50, b=50)
)

# Save the plot as a PDF file if needed
fig.write_image("/Users/gulugulu/repos/PuningAnalysis/results/figures/ENS_stationary.pdf")

# Show the plot
fig.show()



## non-stationary process 


In [8]:
#calculate the accumulation of ENS where the rate matrix change in the middle

def generate_non_stationary_ENS(Q2, pi, t_range):
    """
    Generates the ENS over a range of time points using two different Q matrices before and after a specified time point t1.
    
    Parameters:
    - pi: A numpy array of shape (1, 4) representing the vector pi.
    - Q1, Q2: Two numpy arrays of shape (4, 4) representing the original and new rate matrices, with different stationary distribution.
    - t_range: numpy.linspace defining the start and end of the time range.
    - t1: The time point at which to switch from using Q1 to Q2.
    
    Returns:
    - A list of ENS values for each time point in the range.
    """

    
    ens_values = []
    ens_accumulated = 0  # To keep track of the accumulated ENS value
    
    for t in t_range:
        ens_2 = expected_number_subs(pi, Q2, t) # Update the accumulated ENS at t1 to continue from this point using Q2
        ens_values.append(ens_2)
    
    return ens_values

In [9]:
non_stationary_ENS_value0 = generate_non_stationary_ENS(Q1, test_nst_array3, np.linspace(0, 1, 999))
non_stationary_ENS_value1 = generate_non_stationary_ENS(Q2*0.6, test_nst_array3, np.linspace(0, 1, 999))
non_stationary_ENS_value2 = generate_non_stationary_ENS(Q3, test_nst_array3, np.linspace(0, 1, 999))



In [1]:
fig = go.Figure()

# Add Q3 line
fig.add_trace(go.Scatter(x=np.linspace(0, 10, 999), y=non_stationary_ENS_value0, mode='lines', line = dict(color = '#cd6d2e', width = 4)))

# Add Q5 line

# Update layout for better presentation
fig.update_layout(
    title=None,
    xaxis_title='Conditional R²',
    xaxis=dict(
        title_font=dict(size=25, family='latex', color='black'),
        tickfont=dict(size=20),
    ),
    yaxis_title=r'$ENS$',
    yaxis=dict(
        title_font=dict(size=50),
        tickfont=dict(size=20),
    ),
    template='plotly_white',
    width=800,
    height=450,
    margin=dict(l=100, r=50, t=50, b=50), 
)

# fig.write_image("/Users/gulugulu/repos/PuningAnalysis/results/figures/ENS_Accumulation_non_stationary.png")

# Show the plot
fig.show()



NameError: name 'go' is not defined

In [11]:
#evolution rate at each time point in non-stationary process

def calculate_non_statioanry_mu(Q, pi_0, t):
    """
    Calculate the value of mu prime (μ(t)) for a given substitution rate matrix Q,
    initial nucleotide frequency pi_0, and time t.

    Parameters:
    Q (numpy.ndarray): The substitution rate matrix.
    pi_0 (numpy.ndarray): The initial nucleotide frequency distribution.
    t (float): The time at which to calculate μ(t).

    Returns:
    float: The calculated value of μ(t).
    """
    # Calculate f(t) = pi_0 * exp(Qt)
    f_t = dot(pi_0, expm(Q * t))
    
    # Calculate mu'(t) as the sum of the element-wise product of f(t) and the diagonal of Q
    mu = - dot(f_t,np.diagonal(Q))
    
    return mu

#derivative of evolution rate
def calculate_non_stationary_mu_prime(Q, pi_0, t):
    """
    Correctly calculate the value of mu prime (μ'(t)) based on the provided formula:
    μ'(t) = - pi_0 * Q * exp(Qt) * diag(Q)

    Parameters:
    Q (numpy.ndarray): The substitution rate matrix.
    pi_0 (numpy.ndarray): The initial nucleotide frequency distribution.
    t (float): The time at which to calculate μ'(t).

    Returns:
    numpy.ndarray: The calculated value of μ'(t) as a vector.
    """
    
    # Calculate μ'(t) using the provided formula
    mu_prime_t = -pi_0.dot(Q).dot(expm(Q * t)).dot(np.diagonal(Q))
    
    return mu_prime_t


#Calculate the range of evolution rate and its derivation given a time intervel and initial nucleotide frequency
def mu_mu_prime_range(Q, pi_0, t_range):
    """
    Calculate the range of evolution rate and its derivation 
    given a time intervel and initial nucleotide frequency

    Parameters:
    Q (numpy.ndarray): The substitution rate matrix.
    pi_0 (numpy.ndarray): The initial nucleotide frequency distribution.
    t_range (numpy.linspace): The time interval for calculating the evolutionary rate and its derivative

    Returns:
    lists: The calculated value of μ(t) and μ'(t) over the time inteval as lists.
    """
    mu_range = []
    mu_prime_range = []
    for t in t_range:
        mu = calculate_non_statioanry_mu(Q, pi_0, t)
        mu_prime = calculate_non_stationary_mu_prime(Q, pi_0, t)
        mu_range.append(mu)
        mu_prime_range.append(mu_prime)
    
    return mu_range, mu_prime_range


In [12]:
#test the evolution rate using the testing Q and pi
t_range_large =  np.linspace(0, 1, 100)
t_range_small = np.linspace(0, 0.5, 100)


In [13]:
# mu_range1, mu_prime_range1 = mu_mu_prime_range(Q2*0.6, test_nst_array3, t_range_large)
# mu_range0, mu_prime_range0 = mu_mu_prime_range(Q1, test_nst_array3, t_range_large)
# mu_range2, mu_prime_range2 = mu_mu_prime_range(Q3, test_nst_array3, t_range_large)
# mu_change = abs(np.array(mu_range0) - np.array(mu_range1))

# fig = go.Figure()

# # Add traces
# fig.add_trace(go.Scatter(x=t_range_large, y=mu_range0, mode='lines+markers', name='Evolutionary rate (Q1)'))
# fig.add_trace(go.Scatter(x=t_range_large, y=mu_range1, mode='lines+markers', name='Evolutionary rate (Q2)'))
# # fig.add_trace(go.Scatter(x=t_range_large, y=mu_range2, mode='lines+markers', name='mu_range2 (Q3)'))

# # Update layout
# fig.update_layout(
#     title='<b>Evolutionary Rate in Different Non-Stationary Processes</b>',
#     xaxis_title='Time (t)',
#     yaxis_title='Rate of ENS Accumulation',
#     template='plotly_white',
#     title_font=dict(size=20),
#     xaxis=dict(
#         title_font=dict(size=25),
#         tickfont=dict(size=20),
#     ),
#     yaxis=dict(
#         title_font=dict(size=25),
#         tickfont=dict(size=20),
#     )
# )

# fig.show()




In [124]:
import plotly.graph_objects as go
# Ploting the ENS over time with rate matrix switch in the middle
t_range =  np.linspace(0, 1, 999)
t1 = 0  #The time point that change the rate matrix

ens_values = generate_non_stationary_ENS(Q1*0.2, calculate_stationary_distribution(Q1), t_range)
ens_values2 = generate_non_stationary_ENS(Q2*0.2, calculate_stationary_distribution(Q2), t_range)
# Create the plot
fig = go.Figure()


# Add Q5 line
fig.add_trace(go.Scatter(x=t_range, y=ens_values2, mode='lines', name='Species 1', line=dict(color='#62BE9F', width=4)))

# Add Q3 line
fig.add_trace(go.Scatter(x=t_range, y=ens_values, mode='lines', name='Species 2', line=dict(color='#AC8AB3', width=4)))

# Update layout for better presentation
fig.update_layout(
    title=None,
    title_font=dict(size=20),  # Increase title size, make it bold
    xaxis_title='<b>Time</b>',
    xaxis= dict(
            title_font=dict(size=25, family='Latex', color='black'),
    ),
    yaxis_title=r'$ENS$',
    yaxis=dict(
            title_font=dict(size=25, family='Latex', color='black')
    ),
    legend=dict(
        title=None,
        font=dict(size=16),  # Adjust font size of legend
        x=0.5,               # Center the legend horizontally
        y=-0.2,              # Position the legend below the plot
        xanchor='center',
        yanchor='top',        # Anchor the legend to the top to ensure it's below the plot
        orientation='h'       # Horizontal orientation
    ),
    template='plotly_white',
    margin=dict(l=50, r=50, t=50, b=50), 
    height=500,
    width=800)

# Save the plot as a PDF file if needed
# fig.write_image("/Users/gulugulu/repos/PuningAnalysis/results/figures/ENS_accumulation_two_stationary.pdf")

# Show the plot
fig.show()
fig.write_image("/Users/gulugulu/repos/PuningAnalysis/results/figures/ENS_difference_two_stationary.pdf")



In [125]:
mu_range1, mu_mu_prime_range1 = mu_mu_prime_range(Q1*0.2, calculate_stationary_distribution(Q1), np.linspace(0, 1, 999))
mu_range2, mu_mu_prime_range2 = mu_mu_prime_range(Q2*0.2, calculate_stationary_distribution(Q2), np.linspace(0, 1, 999))
fig = go.Figure()


# Add Q5 line
fig.add_trace(go.Scatter(x=np.linspace(0, 1, 999), y=mu_range2, mode='lines', name='Species 1', line=dict(color='#62BE9F', width=3)))

# Add Q3 line
fig.add_trace(go.Scatter(x=np.linspace(0, 1, 999), y=mu_range1, mode='lines', name='Species 2', line=dict(color='#AC8AB3', width=3)))

fig.add_trace(go.Scatter(x=np.linspace(0, 1, 999), y=np.abs(np.array(mu_range1) - np.array(mu_range2)), mode='lines', name='Difference (Clock violation)', line=dict(color='#ff0b00', width=3, dash='dash')))

# Update layout for better presentation
fig.update_layout(
    title=None,
    title_font=dict(size=20),  # Increase title size, make it bold
    xaxis_title='<b>Time</b>',
    xaxis= dict(
            title_font=dict(size=25, family='Latex', color='black'),
    ),
    yaxis_title=r'$\mu(t)$',
    yaxis=dict(
            title_font=dict(size=25, family='Latex', color='black')
    ),
    legend=dict(
        title=None,
        font=dict(size=16),  # Adjust font size of legend
        x=0.5,               # Center the legend horizontally
        y=-0.2,              # Position the legend below the plot
        xanchor='center',
        yanchor='top',        # Anchor the legend to the top to ensure it's below the plot
        orientation='h'       # Horizontal orientation
    ),
    template='plotly_white',
    margin=dict(l=50, r=50, t=50, b=50), 
    height=500,
    width=800)

fig.update_yaxes(range=[0, 1])

fig.show()
fig.write_image("/Users/gulugulu/repos/PuningAnalysis/results/figures/clock_violation_two_stationary.pdf")

In [14]:
mu_range1, mu_prime_range1 = mu_mu_prime_range(Q1*0.1, test_nst_array3,  np.linspace(0, 10, 999))
fig = go.Figure()

# Add Q3 line
fig.add_trace(go.Scatter(x=np.linspace(0, 10, 999), y=mu_range1, mode='lines', line = dict(color = '#58B8D1', width = 4)))


# Update layout for better presentation
fig.update_layout(
    title=None,
    xaxis_title='<b>Time</b>',
    xaxis=dict(
        title_font=dict(size=25),
        tickfont=dict(size=20),
    ),
    yaxis_title=r'$\mu$',
    yaxis=dict(
        title_font=dict(size=25),
        tickfont=dict(size=20),
    ),
    template='plotly_white',
    width=800,
    height=450,
    margin=dict(l=50, r=50, t=50, b=50), 
)

# Show the plot
fig.write_image("/Users/gulugulu/repos/PuningAnalysis/results/figures/mu_non_stationary.pdf")
fig.show()



In [127]:

non_stationary_ENS_value0 = generate_non_stationary_ENS(Q1*0.2, test_nst_array3, np.linspace(0, 1, 99))
non_stationary_ENS_value1 = generate_non_stationary_ENS(Q2*0.2, test_nst_array3, np.linspace(0, 1, 99))
fig = go.Figure()

# Add Q3 line
fig.add_trace(go.Scatter(x=np.linspace(0, 1, 999), y=non_stationary_ENS_value0, mode='lines', name='Species 1', line=dict(color='#AC8AB3', width=3)))

# Add Q5 line
fig.add_trace(go.Scatter(x=np.linspace(0, 1, 999), y=non_stationary_ENS_value1, mode='lines', name='Species 2', line=dict(color='#62BE9F', width=3)))

# Update layout for better presentation
fig.update_layout(
    title=None,
    title_font=dict(size=20),  # Increase title size, make it bold
    xaxis_title='<b>Time</b>',
    xaxis= dict(
            title_font=dict(size=25, family='Latex', color='black'),
    ),
    yaxis_title=r'$ENS$',
    yaxis=dict(
            title_font=dict(size=25, family='Latex', color='black')
    ),
    legend=dict(
        title=None,
        font=dict(size=16),  # Adjust font size of legend
        x=0.5,               # Center the legend horizontally
        y=-0.2,              # Position the legend below the plot
        xanchor='center',
        yanchor='top',        # Anchor the legend to the top to ensure it's below the plot
        orientation='h'       # Horizontal orientation
    ),
    template='plotly_white',
    margin=dict(l=50, r=50, t=50, b=50), 
    height=500,
    width=800)


fig.show()
fig.write_image("/Users/gulugulu/repos/PuningAnalysis/results/figures/ENS_difference_two_nonstationary.pdf")


In [134]:
mu_range1, mu_mu_prime_range1 = mu_mu_prime_range(Q1*0.2, test_nst_array3, np.linspace(0, 1, 999))
mu_range2, mu_mu_prime_range2 = mu_mu_prime_range(Q2*0.2, test_nst_array3, np.linspace(0, 1, 999))
fig = go.Figure()

# Add Q3 line
fig.add_trace(go.Scatter(x=np.linspace(0, 1, 999), y=mu_range1, mode='lines', name='Species 1', line=dict(color='#AC8AB3', width=3)))

# Add Q5 line
fig.add_trace(go.Scatter(x=np.linspace(0, 1, 999), y=mu_range2, mode='lines', name='Species 2', line=dict(color='#62BE9F', width=3)))

fig.add_trace(go.Scatter(x=np.linspace(0, 1, 999), y=np.abs(np.array(mu_range1) - np.array(mu_range2)), mode='lines', name=r'$\mu(t) \text{ difference}$', line=dict(color='#ff0b00', width=3, dash='dash')))

# Update layout for better presentation
fig.update_layout(
    title=None,
    title_font=dict(size=20),  # Increase title size, make it bold
    xaxis_title='<b>Time</b>',
    xaxis= dict(
            title_font=dict(size=25, family='Latex', color='black'),
    ),
    yaxis_title=r'$\mu(t)$',
    yaxis=dict(
            title_font=dict(size=25, family='Latex', color='black')
    ),
    legend=dict(
        title=None,
        font=dict(size=16, family = 'Latex'),  # Adjust font size of legend
        x=0.5,               # Center the legend horizontally
        y=-0.2,              # Position the legend below the plot
        xanchor='center',
        yanchor='top',        # Anchor the legend to the top to ensure it's below the plot
        orientation='h'       # Horizontal orientation
    ),
    template='plotly_white',
    margin=dict(l=50, r=50, t=50, b=50), 
    height=500,
    width=800)


fig.show()
fig.write_image("/Users/gulugulu/repos/PuningAnalysis/results/figures/Clock_violation_two_nonstationary.pdf")


In [16]:
#calculate the accumulation of ENS where the rate matrix change in the middle
from cogent3.maths.measure import jsd
import scipy

def generate_non_stationary_JAD(Q, pi, t_range):
    """
    Generates the ENS over a range of time points using two different Q matrices before and after a specified time point t1.
    
    Parameters:
    - pi: A numpy array of shape (1, 4) representing the vector pi.
    - Q1, Q2: Two numpy arrays of shape (4, 4) representing the original and new rate matrices, with different stationary distribution.
    - t_range: numpy.linspace defining the start and end of the time range.
    - t1: The time point at which to switch from using Q1 to Q2.
    
    Returns:
    - A list of ENS values for each time point in the range.
    """

    
    JAD_values = []

    for t in t_range:
        p1 = scipy.linalg.expm(Q*t)
        pi_1 = np.dot(pi,p1)
        jad_1 = jsd(pi_1, pi)
        JAD_values.append(jad_1)
    
    return JAD_values

In [17]:
non_stationary_JAD_value0 = generate_non_stationary_JAD(Q1*0.2, test_nst_array3, np.linspace(0, 1, 999))
non_stationary_JAD_value1 = generate_non_stationary_JAD(Q2*0.2, test_nst_array3, np.linspace(0, 1, 999))

fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=np.linspace(0, 1, 999), y=non_stationary_JAD_value0, mode='lines+markers', name='Q1', marker=dict(size=2)))
fig.add_trace(go.Scatter(x=np.linspace(0, 1, 999), y=non_stationary_JAD_value1, mode='lines+markers', name='Q2', marker=dict(size=2)))
# fig.add_trace(go.Scatter(x=np.linspace(0, 1, 999), y=non_stationary_JAD_value2, mode='lines+markers', name='Q3'))

# Update layout
fig.update_layout(
    title='<b>JAD in Different Non-Stationarity process</b>',
    xaxis_title='Time (t)',
    yaxis_title='JAD',
    template='plotly_white',
    title_font=dict(size=20),
    xaxis=dict(
        title_font=dict(size=25),
        tickfont=dict(size=20),
    ),
    yaxis=dict(
        title_font=dict(size=25),
        tickfont=dict(size=20),
    )
)

fig.show()

In [18]:
#calculate the nabla statistic 

def calculate_nabla_statistic(pi_0, Q, t_range):
    """
    Generates the nabla value over a range of time t based on the initial nucleotide freqeuncy distribution pi and subatituiton rate .
    
    Parameters:
    - pi: A numpy array of shape (1, 4)represnting the initial nucleotide freqeuncy distirbution
    - Q1, Q2: A numpy arrays of shape (4, 4) representing the rate matrices.
    - t_range: numpy.linspace defining the start and end of the time range.
    
    Returns:
    - A list of nabla values for each time point in the range.
    """

    nabla_values = []
    for t in t_range:
        pi_deriv = dot(pi_0, dot(Q, expm(Q * t)))
        conv = norm(pi_deriv)
        nabla_values.append(conv)


    return nabla_values
    

In [19]:
def calculate_nabla_statistic(pi_0, Q, t_range):
    """
    Generates the nabla value over a range of time t based on the initial nucleotide freqeuncy distribution pi and subatituiton rate .
    
    Parameters:
    - pi: A numpy array of shape (1, 4)represnting the initial nucleotide freqeuncy distirbution
    - Q1, Q2: A numpy arrays of shape (4, 4) representing the rate matrices.
    - t_range: numpy.linspace defining the start and end of the time range.
    
    Returns:
    - A list of nabla values for each time point in the range.
    """

    nabla_values = []
    for t in t_range:
        pi_deriv = dot(pi_0, dot(Q, expm(Q * t)))
        conv = norm(pi_deriv)
        nabla_values.append(conv)


    return nabla_values

In [20]:
def derivative_norm(Q, pi_0, t_range):

    # Calculate e^(Qt)
    dirivative_values = []
    for t in t_range:
        exp_Qt = expm(Q * t)
        f_t = pi_0.dot(Q.dot(exp_Qt))
        df_t = pi_0.dot(Q.dot(Q.dot(exp_Qt)))
        norm_f_t = np.linalg.norm(f_t)
        derivative = df_t.dot(f_t) / norm_f_t
        dirivative_values.append(derivative)
    
    return dirivative_values

In [21]:
def has_sign_change(lst):
    for i in range(1, len(lst)):
        # Check if the product of adjacent numbers is negative, which indicates a sign change
        if lst[i] * lst[i - 1] < 0:
            return True, i
    return False, None

# for i in range(len(matrices_list)):
#     matrix = matrices_list[i]
#     eigens = np.linalg.eig(matrix)[0]  # Extract eigenvalues
#     # nabla_derivative_list = derivative_norm(matrix, test_nst_array3, np.linspace(0, 2, 100))
#     mu_range, mu_prime_range = mu_mu_prime_range(matrix, np.array([0.25, 0.31, 0.28, 0.15]), np.linspace(0, 5, 100))
#     mu_range_diff = abs(np.max(mu_range) - np.min(mu_range))
#     print(i, mu_range_diff)



In [22]:
mu_range, mu_prime_range = mu_mu_prime_range(matrices_list[6715], test_nst_array3, np.linspace(0, 20, 100))
fig = go.Figure()
# Add Q3 line
fig.add_trace(go.Scatter(x=np.linspace(0, 20, 100), y=mu_range, mode='lines', line = dict(color = '#58B8D1', width = 4)))

# Update axis label font size
fig.update_layout(
    template='plotly_white',
        title=dict(
        text='<b>Non-Monotonic</b>',
        font=dict(size=20),  # Increase title size, make it bold
        x=0.5, y = 0.95,  # Center the title
        xanchor='center',
    ),
    xaxis_title='<b>Time</b>',
    xaxis=dict(
        title_font=dict(size=25),
        tickfont=dict(size=20),
    ),
    yaxis_title=r'$\mu$',
    yaxis=dict(
        title_font=dict(size=25),
        tickfont=dict(size=20),
    ),
    width=700,
    height=500,
    margin=dict(l=20, r=20, t=40, b=20), 
)
# # fig1.update_yaxes(range=[0.05, 0.06])

fig.write_image("/Users/gulugulu/repos/PuningAnalysis/results/figures/evolutionary_rate_change_non-stationary_non-monotonic.pdf")

fig.show()

In [23]:
mu_range1, mu_prime_range1 = mu_mu_prime_range(matrices_list[2831], test_nst_array3, np.linspace(0, 20, 100))
fig = go.Figure()
# Add Q3 line
fig.add_trace(go.Scatter(x=np.linspace(0, 20, 100), y=mu_range1, mode='lines', line = dict(color = '#58B8D1', width = 4)))

# Update axis label font size
fig.update_layout(
    template='plotly_white',
        title=dict(
        text='<b>Monotonic-Decrease</b>',
        font=dict(size=20),  # Increase title size, make it bold
        x=0.5, y = 0.95,  # Center the title
        xanchor='center',
    ),
    xaxis_title='<b>Time</b>',
    xaxis=dict(
        title_font=dict(size=25),
        tickfont=dict(size=20),
    ),
    yaxis_title=r'$\mu$',
    yaxis=dict(
        title_font=dict(size=25),
        tickfont=dict(size=20),
    ),
    width=700,
    height=500,
    margin=dict(l=20, r=20, t=40, b=20), 
)

fig.write_image("/Users/gulugulu/repos/PuningAnalysis/results/figures/evolutionary_rate_change_non-stationary_monotonic_decrease.pdf")

fig.show()







In [24]:
mu_range2, mu_prime_range2 = mu_mu_prime_range(matrices_list[2527], test_nst_array3, np.linspace(0, 20, 100))
fig = go.Figure()
# Add Q3 line
fig.add_trace(go.Scatter(x=np.linspace(0, 20, 100), y=mu_range2, mode='lines', line = dict(color = '#58B8D1', width = 4)))

# Update axis label font size
fig.update_layout(
    template='plotly_white',
        title=dict(
        text='<b>Monotonic-Increase</b>',
        font=dict(size=20),  # Increase title size, make it bold
        x=0.5, y = 0.95,  # Center the title
        xanchor='center',
    ),
    xaxis_title='<b>Time</b>',
    xaxis=dict(
        title_font=dict(size=25),
        tickfont=dict(size=20),
    ),
    yaxis_title=r'$\mu$',
    yaxis=dict(
        title_font=dict(size=25),
        tickfont=dict(size=20),
    ),
    width=700,
    height=500,
    margin=dict(l=20, r=20, t=40, b=20), 
)

fig.write_image("/Users/gulugulu/repos/PuningAnalysis/results/figures/evolutionary_rate_change_non-stationary_monotonic_increase.pdf")

fig.show()








